In [1]:
!pip install torch==2.0.1 torchvision --quiet
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.0.1+cu117.html --quiet
!pip install torch-sparse -f https://data.pyg.org/whl/torch-2.0.1+cu117.html --quiet
!pip install torch-cluster -f https://data.pyg.org/whl/torch-2.0.1+cu117.html --quiet
!pip install torch-spline-conv -f https://data.pyg.org/whl/torch-2.0.1+cu117.html --quiet
!pip install torch-geometric --quiet

In [2]:
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [3]:
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data

In [4]:
random.seed(0)
torch.manual_seed(0)
np.random.seed(0)

In [7]:
import torch
from torch_geometric.data import Data

In [12]:
import random
import torch
import numpy as np
from torch_geometric.data import Data

In [13]:
random.seed(0)
torch.manual_seed(0)
np.random.seed(0)

In [14]:
# Import required libraries
import random
import torch
import numpy as np
from torch_geometric.data import Data

# Set seeds for reproducibility
random.seed(0)
torch.manual_seed(0)
np.random.seed(0)

# Define the graph construction function
def create_graph(users, events, friendships, interactions):
    """
    Constructs a graph from lists of users, events, friendships, and user-event interactions.

    Parameters:
      users         : List of user identifiers (names or indices).
      events        : List of event identifiers (names or indices).
      friendships   : List of tuples representing edges between users.
                      Each tuple is of the form (user_index_a, user_index_b).
      interactions  : List of tuples representing user-event interactions.
                      Each tuple is (user_index, event_index), where the event_index
                      refers to the event in the 'events' list.

    Returns:
      data          : A PyTorch Geometric Data object containing:
                      - x: Node feature matrix (randomly initialized for demonstration).
                      - edge_index: A tensor that encodes graph connectivity.
    """
    # Count the number of user nodes
    num_users = len(users)

    # Count the number of event nodes
    num_events = len(events)

    # Process friendships (user-to-user connections)
    edge_index_friend = friendships

    # Process interactions (user-event connections)
    # Add an offset equal to num_users for event indices to ensure event nodes follow user nodes
    edge_index_interact = [(u, v + num_users) for (u, v) in interactions]

    # Combine both friendship and interaction edges into a single edge list
    combined_edges = edge_index_friend + edge_index_interact

    # Convert the combined edge list to a PyTorch tensor in shape [2, number_of_edges]
    edge_index = torch.tensor(combined_edges, dtype=torch.long).t().contiguous()

    # Create node features as random 16-dimensional vectors
    x = torch.rand(num_users + num_events, 16)

    # Return a PyTorch Geometric Data object encapsulating our graph
    return Data(x=x, edge_index=edge_index)

# ---------------- Sample Usage ----------------

# Define example lists for users and events
users = ["Alice", "Bob", "Charlie"]
events = ["Concert", "Festival"]

# Define sample friendships: bidirectional edges among users (using indices based on the 'users' list)
friendships = [(0, 1), (1, 0), (1, 2), (2, 1)]

# Define sample interactions: each tuple (user_index, event_index)
# For example: Alice (index 0) interacts with "Concert" (index 0) and Charlie (index 2) interacts with "Festival" (index 1)
interactions = [(0, 0), (2, 1)]

# Create the graph data structure by calling our function
graph_data = create_graph(users, events, friendships, interactions)

# Print node features (x) and edge index to verify correct graph construction
print("Node Features (x):")
print(graph_data.x)
print("\nEdge Index:")
print(graph_data.edge_index)


Node Features (x):
tensor([[0.4963, 0.7682, 0.0885, 0.1320, 0.3074, 0.6341, 0.4901, 0.8964, 0.4556,
         0.6323, 0.3489, 0.4017, 0.0223, 0.1689, 0.2939, 0.5185],
        [0.6977, 0.8000, 0.1610, 0.2823, 0.6816, 0.9152, 0.3971, 0.8742, 0.4194,
         0.5529, 0.9527, 0.0362, 0.1852, 0.3734, 0.3051, 0.9320],
        [0.1759, 0.2698, 0.1507, 0.0317, 0.2081, 0.9298, 0.7231, 0.7423, 0.5263,
         0.2437, 0.5846, 0.0332, 0.1387, 0.2422, 0.8155, 0.7932],
        [0.2783, 0.4820, 0.8198, 0.9971, 0.6984, 0.5675, 0.8352, 0.2056, 0.5932,
         0.1123, 0.1535, 0.2417, 0.7262, 0.7011, 0.2038, 0.6511],
        [0.7745, 0.4369, 0.5191, 0.6159, 0.8102, 0.9801, 0.1147, 0.3168, 0.6965,
         0.9143, 0.9351, 0.9412, 0.5995, 0.0652, 0.5460, 0.1872]])

Edge Index:
tensor([[0, 1, 1, 2, 0, 2],
        [1, 0, 2, 1, 3, 4]])


In [15]:
# Import required libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

# Define the Graph Neural Network Model
class EventRecommendationGNN(nn.Module):
    def __init__(self):
        super(EventRecommendationGNN, self).__init__()
        # First GCN layer: transforms input features (16-dim) to hidden features (32-dim)
        self.conv1 = GCNConv(16, 32)
        # Second GCN layer: transforms hidden features (32-dim) back to output features (16-dim)
        self.conv2 = GCNConv(32, 16)

    def forward(self, data):
        """
        Forward pass of the GNN model.

        Parameters:
          data: PyTorch Geometric Data object containing:
                - x: Node feature matrix.
                - edge_index: Edge list tensor representing graph connectivity.

        Returns:
          x: Updated node embeddings after applying two GCN layers.
        """
        x, edge_index = data.x, data.edge_index

        # Apply the first GCN layer followed by ReLU activation
        x = self.conv1(x, edge_index)
        x = F.relu(x)

        # Apply the second GCN layer
        x = self.conv2(x, edge_index)

        return x

In [16]:
def train_model(data, epochs=100, lr=0.01):
    """
    Trains the EventRecommendationGNN model on the given graph data.

    Parameters:
      data: PyTorch Geometric Data object containing graph structure and features.
      epochs: Number of training epochs (default=100).
      lr: Learning rate for optimizer (default=0.01).

    Returns:
      model: Trained EventRecommendationGNN model.
    """
    # Initialize the model
    model = EventRecommendationGNN()

    # Define an optimizer (Adam is commonly used for GNNs)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    # For demonstration purposes, use node features as dummy targets
    target = data.x.clone()

    # Training loop
    for epoch in range(epochs):
        model.train()  # Set model to training mode
        optimizer.zero_grad()  # Reset gradients

        # Forward pass through the model
        output = model(data)

        # Compute loss (Mean Squared Error between output and target)
        loss = F.mse_loss(output, target)

        # Backward pass and optimization step
        loss.backward()
        optimizer.step()

        # Print loss every 10 epochs
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

    return model


In [17]:
def recommend_events(model, data, num_users, k=5):
    """
    Generates event recommendations for each user based on trained embeddings.

    Parameters:
      model: Trained EventRecommendationGNN model.
      data: PyTorch Geometric Data object containing graph structure and features.
      num_users: Number of user nodes in the graph.
      k: Number of top events to recommend per user (default=5).

    Returns:
      recommendations: Indices of top-k recommended events for each user.
      scores: Scores associated with each recommendation.
    """
    model.eval()  # Set model to evaluation mode

    with torch.no_grad():
        # Get updated node embeddings from the trained model
        embeddings = model(data)

    # Separate user and event embeddings
    user_embeddings = embeddings[:num_users]
    event_embeddings = embeddings[num_users:]

    # Compute similarity scores between users and events using dot product
    scores = torch.matmul(user_embeddings, event_embeddings.t())

    # Get top-k events for each user based on similarity scores
    topk = torch.topk(scores, k=k, dim=1)

    return topk.indices, topk.values


In [18]:
# Define example lists for users and events
users = ["Alice", "Bob", "Charlie"]
events = ["Concert", "Festival"]

# Define sample friendships and interactions
friendships = [(0, 1), (1, 0), (1, 2), (2, 1)]
interactions = [(0, 0), (2, 1)]

# Step 1: Create graph data structure
graph_data = create_graph(users, events, friendships, interactions)

# Step 2 & Step 3: Train the GNN model on the graph data
print("Training the GNN Model...")
trained_model = train_model(graph_data)

# Step 4: Generate event recommendations for each user
print("\nGenerating Recommendations...")
recommendations, scores = recommend_events(trained_model, graph_data, num_users=len(users), k=2)

# Display recommendations for each user
for i, rec in enumerate(recommendations):
    print(f"User {users[i]} is recommended events {[events[idx] for idx in rec]}")


Training the GNN Model...
Epoch 10/100, Loss: 0.0869
Epoch 20/100, Loss: 0.0723
Epoch 30/100, Loss: 0.0625
Epoch 40/100, Loss: 0.0554
Epoch 50/100, Loss: 0.0486
Epoch 60/100, Loss: 0.0431
Epoch 70/100, Loss: 0.0393
Epoch 80/100, Loss: 0.0369
Epoch 90/100, Loss: 0.0355
Epoch 100/100, Loss: 0.0347

Generating Recommendations...
User Alice is recommended events ['Concert', 'Festival']
User Bob is recommended events ['Concert', 'Festival']
User Charlie is recommended events ['Concert', 'Festival']


In [19]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

# Define the Graph Neural Network Model
class EventRecommendationGNN(nn.Module):
    def __init__(self):
        super(EventRecommendationGNN, self).__init__()
        # First GCN layer: transforms input features (16-dim) to hidden features (32-dim)
        self.conv1 = GCNConv(16, 32)
        # Second GCN layer: transforms hidden features (32-dim) back to output features (16-dim)
        self.conv2 = GCNConv(32, 16)

    def forward(self, data):
        """
        Forward pass of the GNN model.

        Parameters:
          data: PyTorch Geometric Data object containing:
                - x: Node feature matrix.
                - edge_index: Edge list tensor representing graph connectivity.

        Returns:
          x: Updated node embeddings after applying two GCN layers.
        """
        x, edge_index = data.x, data.edge_index

        # Apply the first GCN layer followed by ReLU activation
        x = self.conv1(x, edge_index)
        x = F.relu(x)

        # Apply the second GCN layer
        x = self.conv2(x, edge_index)

        return x


In [20]:
def train_model(data, epochs=100, lr=0.01):
    """
    Trains the EventRecommendationGNN model on the given graph data.

    Parameters:
      data: PyTorch Geometric Data object containing graph structure and features.
      epochs: Number of training epochs (default=100).
      lr: Learning rate for optimizer (default=0.01).

    Returns:
      model: Trained EventRecommendationGNN model.
    """
    # Initialize the model
    model = EventRecommendationGNN()

    # Define an optimizer (Adam is commonly used for GNNs)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    # For demonstration purposes, use node features as dummy targets
    target = data.x.clone()

    # Training loop
    for epoch in range(epochs):
        model.train()  # Set model to training mode
        optimizer.zero_grad()  # Reset gradients

        # Forward pass through the model
        output = model(data)

        # Compute loss (Mean Squared Error between output and target)
        loss = F.mse_loss(output, target)

        # Backward pass and optimization step
        loss.backward()
        optimizer.step()

        # Print loss every 10 epochs
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

    return model


In [21]:
def recommend_events(model, data, num_users, k=5):
    """
    Generates event recommendations for each user based on trained embeddings.

    Parameters:
      model: Trained EventRecommendationGNN model.
      data: PyTorch Geometric Data object containing graph structure and features.
      num_users: Number of user nodes in the graph.
      k: Number of top events to recommend per user (default=5).

    Returns:
      recommendations: Indices of top-k recommended events for each user.
      scores: Scores associated with each recommendation.
    """
    model.eval()  # Set model to evaluation mode

    with torch.no_grad():
        # Get updated node embeddings from the trained model
        embeddings = model(data)

    # Separate user and event embeddings
    user_embeddings = embeddings[:num_users]
    event_embeddings = embeddings[num_users:]

    # Compute similarity scores between users and events using dot product
    scores = torch.matmul(user_embeddings, event_embeddings.t())

    # Get top-k events for each user based on similarity scores
    topk = torch.topk(scores, k=k, dim=1)

    return topk.indices, topk.values


In [22]:
# Define example lists for users and events
users = ["Alice", "Bob", "Charlie"]
events = ["Concert", "Festival"]

# Define sample friendships and interactions
friendships = [(0, 1), (1, 0), (1, 2), (2, 1)]
interactions = [(0, 0), (2, 1)]

# Step 1: Create graph data structure
graph_data = create_graph(users, events, friendships, interactions)

# Step 2 & Step 3: Train the GNN model on the graph data
print("Training the GNN Model...")
trained_model = train_model(graph_data)

# Step 4: Generate event recommendations for each user
print("\nGenerating Recommendations...")
recommendations, scores = recommend_events(trained_model, graph_data, num_users=len(users), k=2)

# Display recommendations for each user
for i, rec in enumerate(recommendations):
    print(f"User {users[i]} is recommended events {[events[idx] for idx in rec]}")


Training the GNN Model...
Epoch 10/100, Loss: 0.0625
Epoch 20/100, Loss: 0.0551
Epoch 30/100, Loss: 0.0459
Epoch 40/100, Loss: 0.0410
Epoch 50/100, Loss: 0.0365
Epoch 60/100, Loss: 0.0328
Epoch 70/100, Loss: 0.0298
Epoch 80/100, Loss: 0.0275
Epoch 90/100, Loss: 0.0259
Epoch 100/100, Loss: 0.0249

Generating Recommendations...
User Alice is recommended events ['Concert', 'Festival']
User Bob is recommended events ['Concert', 'Festival']
User Charlie is recommended events ['Concert', 'Festival']


In [23]:
def train_model(data, epochs=100, lr=0.01):
    """
    Trains the EventRecommendationGNN model on the given graph data.

    Parameters:
      data: PyTorch Geometric Data object containing graph structure and features.
      epochs: Number of training epochs (default=100).
      lr: Learning rate for optimizer (default=0.01).

    Returns:
      model: Trained EventRecommendationGNN model.
    """
    # Initialize the GNN model
    model = EventRecommendationGNN()

    # Define an optimizer (Adam is commonly used for GNNs)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    # For demonstration purposes, use node features as dummy targets
    target = data.x.clone()

    # Training loop
    for epoch in range(epochs):
        model.train()  # Set model to training mode
        optimizer.zero_grad()  # Reset gradients

        # Forward pass through the model
        output = model(data)

        # Compute loss (Mean Squared Error between output and target)
        loss = F.mse_loss(output, target)

        # Backward pass and optimization step
        loss.backward()
        optimizer.step()

        # Print loss every 10 epochs
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

    return model


In [24]:
# Train the GNN Model
print("Training the GNN Model...")
trained_model = train_model(graph_data, epochs=100, lr=0.01)


Training the GNN Model...
Epoch 10/100, Loss: 0.0858
Epoch 20/100, Loss: 0.0633
Epoch 30/100, Loss: 0.0545
Epoch 40/100, Loss: 0.0510
Epoch 50/100, Loss: 0.0472
Epoch 60/100, Loss: 0.0436
Epoch 70/100, Loss: 0.0397
Epoch 80/100, Loss: 0.0355
Epoch 90/100, Loss: 0.0319
Epoch 100/100, Loss: 0.0291


In [25]:
def recommend_events(model, data, num_users, k=5):
    """
    Generates event recommendations for each user based on trained embeddings.

    Parameters:
      model: Trained EventRecommendationGNN model.
      data: PyTorch Geometric Data object containing graph structure and features.
      num_users: Number of user nodes in the graph.
      k: Number of top events to recommend per user (default=5).

    Returns:
      recommendations: Indices of top-k recommended events for each user.
      scores: Scores associated with each recommendation.
    """
    model.eval()  # Set model to evaluation mode

    with torch.no_grad():
        # Get updated node embeddings from the trained model
        embeddings = model(data)

    # Separate user and event embeddings
    user_embeddings = embeddings[:num_users]
    event_embeddings = embeddings[num_users:]

    # Compute similarity scores between users and events using dot product
    scores = torch.matmul(user_embeddings, event_embeddings.t())

    # Get top-k events for each user based on similarity scores
    topk = torch.topk(scores, k=k, dim=1)

    return topk.indices, topk.values


In [26]:
# Generate event recommendations for each user
print("\nGenerating Recommendations...")
recommendations, scores = recommend_events(trained_model, graph_data, num_users=len(users), k=2)

# Display recommendations for each user
for i, rec in enumerate(recommendations):
    print(f"User {users[i]} is recommended events {[events[idx] for idx in rec]}")



Generating Recommendations...
User Alice is recommended events ['Concert', 'Festival']
User Bob is recommended events ['Concert', 'Festival']
User Charlie is recommended events ['Concert', 'Festival']
